# Notebook pro trénink s destilací nad datasetem CIFAR100
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR100, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

## Import knihoven a definice metod

In [1]:
from transformers import Trainer, EarlyStoppingCallback
from torch.utils.data import ConcatDataset, DataLoader
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
dataset_part = base.get_dataset_part()
DATASET = "cifar100"

Inicializovaný MobileNetV2.

In [3]:
base.reset_seed()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Provedení transformací nad datasetem.

In [5]:
transform = base.base_transforms()

train = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=transform)
test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=transform)


In [6]:
augment_transform = base.aug_transforms()

train_aug = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)

In [7]:
train_part_cpu = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=transform, device="cpu")
cpu_data_loader = DataLoader(train_part_cpu, batch_size=1, shuffle=False)
train_part_gpu = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=transform, device="cuda")
gpu_data_loader = DataLoader(train_part_gpu, batch_size=1, shuffle=False)

In [8]:
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
print(len(train_aug))
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

25912


### Standardní trénink náhodně inicializovaného modelu. 

In [9]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random-base_aug", logging_dir=f"~/logs/{DATASET}/random-base_aug", lr=0.0005, weight_decay=0.008, warmup_steps=5, epochs=20)
model = base.get_random_init_mobilenet(100)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,4.012100,3.491575,0.145900,0.145650,0.145900,0.111652
2,3.221500,2.753546,0.282600,0.281917,0.282600,0.253578
3,2.720300,2.431204,0.357300,0.359042,0.357300,0.338813
4,2.377900,2.163932,0.414600,0.411753,0.414600,0.395408
5,2.107700,1.998699,0.452500,0.461214,0.452500,0.439678
6,1.886500,1.858586,0.489500,0.496367,0.489500,0.479106
7,1.691900,1.764659,0.514000,0.522273,0.514000,0.505659
8,1.516100,1.706783,0.529700,0.539177,0.529700,0.525424
9,1.351300,1.659283,0.540500,0.551045,0.540500,0.538190
10,1.199900,1.627684,0.548900,0.551294,0.548900,0.545230


TrainOutput(global_step=9270, training_loss=1.5059889956971202, metrics={'train_runtime': 2087.5688, 'train_samples_per_second': 631.471, 'train_steps_per_second': 4.934, 'total_flos': 2.520215532845531e+18, 'train_loss': 1.5059889956971202, 'epoch': 18.0})

In [12]:
model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [13]:
trainer.evaluate(test)

{'eval_loss': 1.7099978923797607,
 'eval_accuracy': 0.5647,
 'eval_precision': 0.5753105173095028,
 'eval_recall': 0.5647,
 'eval_f1': 0.5652796275462485,
 'eval_runtime': 13.7991,
 'eval_samples_per_second': 724.683,
 'eval_steps_per_second': 5.725,
 'epoch': 18.0}

In [14]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/random-base_aug.pth")

## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

### Trénink náhodně inicializovaného modelu s pomocí destilace znalostí

In [15]:
base.reset_seed()

In [16]:
student_model = base.get_random_init_mobilenet(100)

In [17]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random-distill_aug", logging_dir=f"~/logs/{DATASET}/random-distill_aug", remove_unused_columns=False, epochs=20, lr=0.00055, lambda_param=.7, temp=6)

In [18]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.075500,2.773163,0.170600,0.186867,0.170600,0.125151
2,2.588100,2.309701,0.294400,0.310505,0.294400,0.253519
3,2.256600,2.057659,0.376500,0.400820,0.376500,0.356979
4,2.012400,1.855225,0.433600,0.447542,0.433600,0.411304
5,1.822600,1.732531,0.474800,0.490015,0.474800,0.457242
6,1.674400,1.623069,0.502000,0.511607,0.502000,0.487226
7,1.541300,1.536804,0.522300,0.533391,0.522300,0.511381
8,1.428000,1.465016,0.549400,0.554212,0.549400,0.540067
9,1.320900,1.443941,0.554900,0.567294,0.554900,0.549770
10,1.228400,1.378345,0.571100,0.572900,0.571100,0.565444


TrainOutput(global_step=10300, training_loss=1.370963890297899, metrics={'train_runtime': 2324.7587, 'train_samples_per_second': 567.044, 'train_steps_per_second': 4.431, 'total_flos': 2.800239480939479e+18, 'train_loss': 1.370963890297899, 'epoch': 20.0})

In [20]:
student_model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [21]:
trainer.evaluate(test)

{'eval_loss': 1.1964892148971558,
 'eval_accuracy': 0.5968,
 'eval_precision': 0.6129856346358608,
 'eval_recall': 0.5968,
 'eval_f1': 0.5981585844379341,
 'eval_runtime': 13.1559,
 'eval_samples_per_second': 760.115,
 'eval_steps_per_second': 6.005,
 'epoch': 20.0}

In [22]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/random-distill_aug.pth")

## Získání inicializovaného MobileNetV2 modelu

In [23]:
base.reset_seed()

In [24]:
model_pretrained = base.get_mobilenet(100)

In [25]:
print(model_pretrained)

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [26]:
model_pretrained = base.freeze_model(model_pretrained)

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/head-base_aug", logging_dir=f"~/logs/{DATASET}/head-base_aug", epochs=20, lr=0.001, weight_decay=0.005, warmup_steps=20)

In [28]:
trainer = Trainer(
    model=model_pretrained,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.652600,1.845513,0.518000,0.542144,0.518000,0.513308
2,1.965000,1.743547,0.531900,0.553089,0.531900,0.529704
3,1.827700,1.706149,0.539800,0.564763,0.539800,0.538563
4,1.755400,1.676541,0.550300,0.568118,0.550300,0.548967
5,1.705800,1.697658,0.544300,0.563179,0.544300,0.540595
6,1.666900,1.694501,0.544800,0.562332,0.544800,0.541697
7,1.637800,1.677576,0.546500,0.567442,0.546500,0.545399


TrainOutput(global_step=3605, training_loss=1.8873015891827756, metrics={'train_runtime': 561.7114, 'train_samples_per_second': 2346.828, 'train_steps_per_second': 18.337, 'total_flos': 9.800838183288177e+17, 'train_loss': 1.8873015891827756, 'epoch': 7.0})

In [30]:
model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [31]:
trainer.evaluate(test)

{'eval_loss': 1.673932433128357,
 'eval_accuracy': 0.5476,
 'eval_precision': 0.5683108863736596,
 'eval_recall': 0.5476000000000001,
 'eval_f1': 0.5470175473783051,
 'eval_runtime': 12.6112,
 'eval_samples_per_second': 792.946,
 'eval_steps_per_second': 6.264,
 'epoch': 7.0}

In [32]:
torch.save(model_pretrained.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/head-base_aug.pth")

### Trénink inicializovaného MobileNetV2

In [69]:
base.reset_seed()

In [70]:
model_pretrained_whole = base.get_mobilenet(100)

In [71]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-base_aug", logging_dir=f"~/logs/{DATASET}/pretrained-base_aug", epochs=20, lr=0.0004, weight_decay=0.008, warmup_steps=10)

In [72]:
trainer = Trainer(
    model=model_pretrained_whole,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [73]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.503800,1.003019,0.705900,0.721646,0.705900,0.702884
2,0.684100,0.864370,0.747200,0.759304,0.747200,0.745554
3,0.426500,0.881176,0.754900,0.766942,0.754900,0.752936
4,0.277100,0.943901,0.748000,0.762144,0.748000,0.748892
5,0.187700,0.975006,0.758600,0.768252,0.758600,0.757383
6,0.134300,1.006056,0.754000,0.761981,0.754000,0.753194
7,0.097400,1.048559,0.760800,0.770917,0.760800,0.761292
8,0.077300,1.029750,0.769100,0.776869,0.769100,0.768439
9,0.060200,1.088217,0.767700,0.775303,0.767700,0.768304
10,0.045600,1.117871,0.764600,0.770101,0.764600,0.764286


TrainOutput(global_step=5665, training_loss=0.320696266288791, metrics={'train_runtime': 1266.9369, 'train_samples_per_second': 1040.494, 'train_steps_per_second': 8.13, 'total_flos': 1.5401317145167135e+18, 'train_loss': 0.320696266288791, 'epoch': 11.0})

In [74]:
model_pretrained_whole.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [75]:
trainer.evaluate(test)

{'eval_loss': 1.0523370504379272,
 'eval_accuracy': 0.7647,
 'eval_precision': 0.7730122282487011,
 'eval_recall': 0.7646999999999998,
 'eval_f1': 0.7644034733518786,
 'eval_runtime': 16.1603,
 'eval_samples_per_second': 618.801,
 'eval_steps_per_second': 4.889,
 'epoch': 11.0}

In [40]:
torch.save(model_pretrained_whole.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/pretrained-base_aug.pth")

## Trénink s pomocí destilace znalostí inicializovaného MobileNetV2

### Trénink inicializovaného modelu - pouze klasifikační hlavy s pomocí destilace

In [41]:
base.reset_seed()

In [42]:
student_model_pretrained = base.get_mobilenet(100)

In [43]:
student_model_pretrained = base.freeze_model(student_model_pretrained)

In [44]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/head-distill_aug", logging_dir=f"~/logs/{DATASET}/head-distill_aug", remove_unused_columns=False, epochs=20, lr=0.0012, weight_decay=.005, warmup_steps=15, lambda_param=.4, temp=5)

In [45]:
trainer = base.DistilTrainer(
    student_model=student_model_pretrained,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [46]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.664300,2.116916,0.523600,0.542366,0.523600,0.515046
2,2.244200,2.059310,0.534900,0.548560,0.534900,0.527563
3,2.175700,2.038093,0.539900,0.565486,0.539900,0.535931
4,2.141400,2.010734,0.553600,0.572997,0.553600,0.549893
5,2.119300,2.001260,0.551900,0.565355,0.551900,0.546022
6,2.101200,2.023759,0.545100,0.567735,0.545100,0.544335
7,2.088700,1.991565,0.552400,0.569127,0.552400,0.549587


TrainOutput(global_step=3605, training_loss=2.2192619165005203, metrics={'train_runtime': 612.2123, 'train_samples_per_second': 2153.24, 'train_steps_per_second': 16.824, 'total_flos': 9.800838183288177e+17, 'train_loss': 2.2192619165005203, 'epoch': 7.0})

In [47]:
student_model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [48]:
trainer.evaluate(test)

{'eval_loss': 1.9305979013442993,
 'eval_accuracy': 0.5429,
 'eval_precision': 0.5620144425218659,
 'eval_recall': 0.5428999999999999,
 'eval_f1': 0.5398014717487571,
 'eval_runtime': 12.6319,
 'eval_samples_per_second': 791.645,
 'eval_steps_per_second': 6.254,
 'epoch': 7.0}

In [49]:
torch.save(student_model_pretrained.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/head-distill_aug.pth")

### Trénink inicializovaného modelu s pomocí destilace

In [62]:
base.reset_seed()

In [63]:
student_model_pretrained_whole = base.get_mobilenet(100)

In [64]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-distill_aug", logging_dir=f"~/logs/{DATASET}/pretrained-distill_aug", remove_unused_columns=False, epochs=20, lr=0.0005, weight_decay=.005, warmup_steps=5, lambda_param=.6, temp=6.5)

In [65]:
trainer = base.DistilTrainer(
    student_model=student_model_pretrained_whole.to(device),
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [66]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.301100,0.946234,0.710700,0.723578,0.710700,0.706476
2,0.730500,0.820093,0.746100,0.758291,0.746100,0.744260
3,0.542600,0.771098,0.759900,0.771935,0.759900,0.759221
4,0.426500,0.757994,0.765800,0.779472,0.765800,0.766924
5,0.349500,0.760088,0.762600,0.772940,0.762600,0.762716
6,0.294400,0.736535,0.778700,0.791073,0.778700,0.779681
7,0.256000,0.748447,0.768900,0.781456,0.768900,0.770089
8,0.228200,0.708099,0.781500,0.791045,0.781500,0.781683
9,0.205900,0.718650,0.775500,0.786709,0.775500,0.776588
10,0.187300,0.686474,0.786200,0.792348,0.786200,0.786376


TrainOutput(global_step=6695, training_loss=0.3850760291101685, metrics={'train_runtime': 1496.6729, 'train_samples_per_second': 880.78, 'train_steps_per_second': 6.882, 'total_flos': 1.8201556626106614e+18, 'train_loss': 0.3850760291101685, 'epoch': 13.0})

In [67]:
student_model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [68]:
trainer.evaluate(test)

{'eval_loss': 0.647940993309021,
 'eval_accuracy': 0.782,
 'eval_precision': 0.7897126741699193,
 'eval_recall': 0.782,
 'eval_f1': 0.7825311803661508,
 'eval_runtime': 12.2037,
 'eval_samples_per_second': 819.425,
 'eval_steps_per_second': 6.473,
 'epoch': 13.0}

In [57]:
torch.save(student_model_pretrained_whole.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/pretrained-distill_aug.pth")

In [58]:
base.count_parameters(student_model_pretrained_whole)

model size: 9.103MB.
Total Trainable Params: 2351972.


,Modules,Parameters
0,mobilenet_v2.conv_stem.first_conv.convolution....,864
1,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
2,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
3,mobilenet_v2.conv_stem.conv_3x3.convolution.we...,288
4,mobilenet_v2.conv_stem.conv_3x3.normalization....,32
...,...,...
153,mobilenet_v2.conv_1x1.convolution.weight,409600
154,mobilenet_v2.conv_1x1.normalization.weight,1280
155,mobilenet_v2.conv_1x1.normalization.bias,1280
156,classifier.weight,128000


In [59]:
base.count_parameters(student_model_pretrained)

model size: 9.103MB.
Total Trainable Params: 128100.


,Modules,Parameters
0,classifier.weight,128000
1,classifier.bias,100


In [60]:
cpu_benchmark = base.BenchMarkRunner(student_model_pretrained_whole, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  28.52 ms
  1 measurement, 1000 runs , 4 threads


In [61]:
gpu_benchmark = base.BenchMarkRunner(student_model_pretrained_whole, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  8.88 ms
  1 measurement, 1000 runs , 4 threads
